# LightSSS：轻量级仿真快照

打印完整波形很慢，且浪费磁盘空间，实际的出错点往往在 difftest 框架报错（体系结构状态改变）附近。

LightSSS 允许在不启用波形的情况下运行仿真，直到出错以后再恢复到出错前一小段时间的状态，重新仿真并记录波形。

具体原理上，其使用 `fork()` 系统调用记录进程的快照，fork 出的子进程处于休眠状态。直到检测到 bug 时，子进程会被唤醒并在启用波形输出的状态下重新运行。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd $NOOP_HOME

rm -f ./build/*.vcd

${READY2RUN_HOME}/emu-alu-err \
    -i ${READY2RUN_HOME}/hello-riscv64-xs.bin \
    --diff $NEMU_HOME/build/riscv64-nemu-interpreter-so \
    --enable-fork \
    2> /dev/null || true

echo -n "Dump wave: "
realpath ./build/*.vcd

如果您看到“the oldest checkpoint start to dump wave and dump nemu log...”，则表示 LightSSS 已生效。此后，模拟将从现有的最新快照重新开始并记录波形。